### 1. Dependencies

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import DataParallel

import time
import os
import numpy as np
import json
import cv2
from PIL import Image, ImageOps
import random
from tqdm import tqdm
import operator
import itertools
from scipy.io import  loadmat
import logging
from scipy import signal

from utils import data_transforms
from utils import get_paste_kernel, kernel_map
from utils_logging import setup_logger

### 2. Choose between Recasens or GazeNet

- Idea is you can just swap 
models.recasens, dataloader.recasens, training.train_recasens, etc...
- with the following
models.gazenet, dataloader.gazenet, training.train_gazenet

In [2]:
from models.gazenet import GazeNet
from models.__init__ import save_checkpoint, resume_checkpoint
from dataloader.gazenet import GooDataset, GazeDataset
from training.train_gazenet import train, test, GazeOptimizer

In [3]:
# Logger will save the training and test errors to a .log file 
logger = setup_logger(name='first_logger', 
                      log_dir ='./logs/',
                      log_file='train_gazenet_gooreal_.log',
                      log_format = '%(asctime)s %(levelname)s %(message)s',
                      verbose=True)

### 3. Dataloaders
- Choose between GazeDataset (Gazefollow dataset) or GooDataset (GooSynth/GooReal)
- Set paths to image directories and pickle paths. For Gazefollow, images_dir and test_images_dir should be the same and both lead to the path containing the train and test folders.

In [4]:
# Dataloaders for GazeFollow
batch_size=32
workers=12
testbatchsize=16

images_dir = '/home/eee198/Documents/datasets/GazeFollowData/'
pickle_path = '/home/eee198/Documents/datasets/GazeFollowData/train_annotations.mat'
test_images_dir = '/home/eee198/Documents/datasets/GazeFollowData/'
test_pickle_path = '/home/eee198/Documents/datasets/GazeFollowData/test_annotations.mat'

train_set = GazeDataset(images_dir, pickle_path, 'train')
train_data_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers)

val_set = GazeDataset(test_images_dir, test_pickle_path, 'test')
test_data_loader = torch.utils.data.DataLoader(val_set, batch_size=testbatchsize, num_workers=workers, shuffle=False)

In [4]:
# Dataloaders for GOO-Synth
batch_size=32
workers=12
testbatchsize=32

images_dir = '/hdd/HENRI/goosynth/1person/GazeDatasets/'
pickle_path = '/hdd/HENRI/goosynth/picklefiles/trainpickle2to19human.pickle'
test_images_dir = '/hdd/HENRI/goosynth/test/'
test_pickle_path = '/hdd/HENRI/goosynth/picklefiles/testpickle120.pickle'

train_set = GooDataset(images_dir, pickle_path, 'train')
train_data_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers)

val_set = GooDataset(test_images_dir, test_pickle_path, 'test')
test_data_loader = torch.utils.data.DataLoader(val_set, batch_size=testbatchsize, num_workers=workers, shuffle=False)

==> Number of Images: 172800
==> Number of Images: 19200


In [4]:
# Dataloaders for GOO-Real
batch_size=32
workers=12
testbatchsize=32

images_dir = '/home/eee198/Documents/datasets/GOOReal/finalrealdatasetImgsV2/'
pickle_path = '/home/eee198/Documents/datasets/GOOReal/oneshotrealhumans.pickle'
test_images_dir = '/home/eee198/Documents/datasets/GOOReal/finalrealdatasetImgsV2/'
test_pickle_path = '/home/eee198/Documents/datasets/GOOReal/testrealhumans.pickle'

train_set = GooDataset(images_dir, pickle_path, 'train')
train_data_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers)

val_set = GooDataset(test_images_dir, test_pickle_path, 'test')
test_data_loader = torch.utils.data.DataLoader(val_set, batch_size=testbatchsize, num_workers=workers, shuffle=False)

==> Number of Images: 2451
==> Number of Images: 2156


### 4. Load Model and Set Training Hyperparameters
- For Gazefollow, the model requires the alexnet_places365 pretrained model, provided here: https://urlzs.com/ytKK3
- When resuming training, set to True and set the resume_path for the saved model.
- Here, logging module is initialized (logger) to save training and testing errors.

In [5]:
# Loads model
net = GazeNet()
net.cuda()

# Hyperparameters
start_epoch = 25
max_epoch = 30
learning_rate = 1e-4

# Initializes Optimizer
gaze_opt = GazeOptimizer(net, learning_rate)
optimizer = gaze_opt.getOptimizer(start_epoch)

# Is training resumed? If so, set the resume_path and set flag to True
# This can also be used to evaluate a model 
resume_training = True
resume_path = './saved_models/goosynth_pretrain/lian.pth.tar'
if resume_training :
    net, optimizer, _ = resume_checkpoint(net, optimizer, resume_path)
    scores = test(net, test_data_loader,logger, save_output=True)
    #print('L2 Distance: ', scores[1])

=> loading checkpoint './saved_models/goosynth_pretrain/lian.pth.tar'


  0%|          | 0/68 [00:00<?, ?it/s]

=> loaded checkpoint './saved_models/goosynth_pretrain/lian.pth.tar' (epoch 24)


loss: 0.13326, 0.56249, 0.69574
100%|██████████| 68/68 [00:53<00:00,  1.27it/s]
average error: [0.7702479087857104, 0.2752074334177219, 49.5848842313793]


In [6]:
scores = test(net, test_data_loader,logger, save_output=True)

100%|██████████| 68/68 [00:52<00:00,  1.29it/s]
average error: [0.579888081634661, 0.3513668198610138, 75.51560973114206]


### 5. Training the Model
- Determine in which epochs do you want to save the model, as you might not want to save every epoch
- Training and test errors can be accessed in the logs directory set up earlier

In [8]:
best_l2 = np.inf

for epoch in range(26, 30):
    
    # Update optimizer
    optimizer = gaze_opt.getOptimizer(epoch)

    # Train model
    train(net, train_data_loader, optimizer, epoch, logger)
    
    # Evaluate model
    #scores = test(net, test_data_loader, logger)
    
    # Save model+optimizer with best L2 Score
    #if scores[1] < best_l2:
    #    best_l2 = scores[1]
    #    save_path = './saved_models/gazenet_gooreal_notrain/'
    #    save_checkpoint(net, optimizer, 420, save_path)

100%|██████████| 77/77 [01:03<00:00,  1.21it/s]


In [9]:
test(net, test_data_loader,logger, save_output=True)

100%|██████████| 68/68 [00:55<00:00,  1.22it/s]
average error: [0.8884825397733727, 0.17205015173593233, 33.337276480865654]


[0.8884825397733727, 0.17205015173593233, 33.337276480865654]